# Requirements
- 1)a -- demean w1, b -- demean w2; 
- 2)a -- 对于之前生成的csv文件100行 取使得n of inequality为max的beta值; b -- 100行的average (n of inequality 也是average); CI 用同样的方式(即如果beta是 arg max 那CI也是用arg max， 如果beta是mean, CI 也是mean);
- 3）bound：+-100;+-500;+-1000

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
import time
import multiprocessing as mp
from multiprocessing import (Pool, Process, Queue, Lock,
                             cpu_count,  current_process)
import random
random.seed(10)
np.set_printoptions(suppress = True)

# Demean

df_w1
- var1 = Relationship_Dum(demean)
- var2 = mi_to_zcta5(demean)
- var3 = FinTechIndicator(demean) * rating_avg(demean)
- var4 = FinTechIndicator(demean) * minority_yelp(demean)


df_w2
- var1 = Relationship_Dum
- var2 = mi_to_zcta5
- var3 = FinTechIndicator(demean) * rating_avg(demean)
- var4 = FinTechIndicator(demean) * minority_yelp(demean)

In [15]:
def Exchange_pairs(df_matched, df_unmatched):
    # t1 = time.time()
    
    df_matched_column = df_matched.columns
    df_matched.columns = df_matched_column + '1m'
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id', 'lender_id1m')
    df1 = pd.merge(df_matched, df_unmatched, on = 'lender_id1m', how = 'inner')
    
    l = df1.columns[:-5].append([df1.columns[-5:] + '1um'])
    df1.columns = l
    df_matched.columns = df_matched_column
    df2 = pd.merge(df_matched, df1, left_on = 'loan_id', right_on = 'loan_id1um', how = 'inner') 
    
    ll = (df2.columns[:6]+'2m').append(df2.columns[6:])
    df2.columns = ll
    df_unmatched.columns = df_unmatched.columns.str.replace('lender_id1m', 'lender_id')
    df3 = pd.merge(df_unmatched, df2, left_on = ['lender_id','loan_id'], right_on = ['lender_id2m','loan_id1m'], how = 'inner')
    lll = (df3.columns[:6]+'2um').append(df3.columns[6:])
    df3.columns = lll
    
    df_keep = pd.DataFrame()
    for i in range(1, 5):
        name = "value" + str(i)
        df_keep[name] = df3["var"+str(i)+"1m"] + df3["var"+str(i)+"2m"] - df3["var"+str(i)+"1um"] - df3["var"+str(i)+"2um"]
    # t2 = time.time()
    # print("Running time: ", t2-t1)
    return df_keep

In [19]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_sample = df_w2
'''
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]

df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2021].iloc[:,: 6]

df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2021].iloc[:,: 6]



df_exchange_pairs = Exchange_pairs(df_matched, df_unmatched)
'''
'''
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2020]
df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020]
# df_keep = Exchange_pairs(df_matched, df_unmatched)
'''

/opt/apps/rhel8/Anaconda3-2021.05/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/apps/rhel8/Anaconda3-2021.05/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/opt/apps/rhel8/Anaconda3-2021.05/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

'\ndf_matched = df_sample[df_sample["match"] == 1]\ndf_unmatched = df_sample[df_sample["match"] == 0]\ndf_matched.drop(["match"], axis = 1, inplace = True) \ndf_unmatched.drop(["match"], axis = 1, inplace = True) \ndf_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]\ndf_matched = df_matched[df_matched["yearapproved"] == 2020]\ndf_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]\ndf_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020]\n# df_keep = Exchange_pairs(df_matched, df_unmatched)\n'

In [17]:
df_largest = df_matched.groupby("USPS_ZIP_PREF_STATE", group_keys = False).apply(lambda x: x["loan_id"].unique().shape[0]).sort_values(ascending = False)[0:10]
largest_list = list(df_largest.index)
largest_list

['NY']

In [18]:
df_sample = df_sample[df_sample["USPS_ZIP_PREF_STATE"].isin(largest_list)]
df_sample

,lender_id,loan_id,match,var1,var2,var3,var4,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,yearapproved
87862,814.0,19445.0,1.0,0.0,11.481559,-0.139875,0.045413,FISHERS ISLAND,NY,2020.0
156347,561.0,29072.0,0.0,0.0,123.103606,-0.024243,0.045413,NEW YORK,NY,2020.0
156348,561.0,77093.0,0.0,0.0,123.103606,0.160768,0.045413,NEW YORK,NY,2020.0
156349,561.0,49444.0,0.0,0.0,123.103606,-0.024243,0.045413,NEW YORK,NY,2020.0
156350,561.0,77470.0,0.0,0.0,123.103606,-0.024243,0.045413,NEW YORK,NY,2020.0
...,...,...,...,...,...,...,...,...,...,...
599443,3278.0,36138.0,0.0,0.0,174.711647,-0.082059,0.045413,ELMIRA,NY,2020.0
599444,697.0,36138.0,0.0,0.0,1279.236655,-0.082059,0.045413,ELMIRA,NY,2020.0
599445,697.0,19525.0,0.0,0.0,1279.236655,0.322653,-0.070219,ELMIRA,NY,2020.0
599446,3923.0,36138.0,0.0,0.0,0.000000,0.627596,-0.347326,ELMIRA,NY,2020.0


In [20]:
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
df_keep = Exchange_pairs(df_matched, df_unmatched)

/opt/apps/rhel8/Anaconda3-2021.05/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [21]:
def Fox_func(df_keep, num, ratio):
    
    
    df_keep_copy = df_keep.copy() # Make a copy
    
    # df_keep.drop(["value3"], axis = 1, inplace = True) 
    # df_keep.drop(["value4"], axis = 1, inplace = True) 
    
    # df_keep.drop(["value2"], axis = 1) 
    # df_keep.drop(["value3"], axis = 1) 

    def objectfunc(beta, df = df_keep_copy):
        return -sum(df.dot(beta) >=0 )
    t1 = time.time()
    # bounds = [(1, 1.0000000001), (-100, 100), (-100, 100), (-100, 100)]
    # bounds = [(-1.000000001, -1), (-500, 500), (-500, 500)]
    
    # bounds = [(-500, 500), (-500, 500), (-500, 500)]
    # bounds = [(-100, 100), (-1.000000001, -1), (-100, 100), (-100, 100)]
    # bounds = [(-1500, 1500), (-1.000000001, -1), (-1500, 1500), (-1500, 1500)]
    bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100)]
    # bounds = [(-1000, 1000), (-1.000000001, -1), (-1000, 1000), (-1000, 1000)]
    result = differential_evolution(objectfunc, bounds)
    # print(result)

    r = np.append(result.x, round(-result.fun))
    
    # t2 = time.time()
    # print("Differential Evolution time: ", t2 - t1)
    # print("The number of inequalities satisfied is")
    # print(round(-result.fun))
    # Simulation(num, ratio, df_matched, df_unmatched)
    return r


In [23]:
%%time
df_result = pd.DataFrame()
for i in range(0, 1):
    print(i)
    result = Fox_func(df_keep, 500, 0.1)
    df_result = df_result.append([list(result)], ignore_index = True)
df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]

0
CPU times: user 3h 29min 46s, sys: 13min 53s, total: 3h 43min 39s
Wall time: 19min 8s


In [36]:
def Fox_opt(num):
    df_result = pd.DataFrame()
    for i in range(0, num):
        print(i)
        result = Fox_func(df_keep, 500, 0.1)
        df_result = df_result.append([list(result)], ignore_index = True)
    df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]
    return df_result

In [34]:
num_cores = cpu_count()
pool = Pool(num_cores)

86

In [40]:
%%time
if __name__ == '__main__':
    num_cores = cpu_count()
    pool = Pool(num_cores)
    result = pool.apply_async(Fox_opt, (1, )).get()
    pool.close()
    pool.join()

0
CPU times: user 166 ms, sys: 2.09 s, total: 2.26 s
Wall time: 8min 38s


In [44]:
%%time
if __name__ == '__main__':
    num_cores = cpu_count()
    pool = Pool(num_cores)
    result = pool.apply_async(Fox_opt, (2, )).get()
    pool.close()
    pool.join()

0
1
CPU times: user 182 ms, sys: 1.95 s, total: 2.13 s
Wall time: 15min 37s


In [46]:
result

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
0,1.0,-1.089177,-7.844220,93.512202,5795814.0
1,1.0,-1.379305,-22.202177,92.133427,5776903.0


In [27]:
# Don't use argmax or idxmax since if the maximum is achieved in multiple locations, only the first is returned.
num_mean = df_result.mean()
num_max = df_result["Number of of inequalities satisfied"].max()
df_result[df_result["Number of of inequalities satisfied"] == num_max]
df_test = df_result
df_test = df_test.append(df_result[df_result["Number of of inequalities satisfied"] == num_max])
df_test = df_test.append(df_result.mean(axis = 0).rename("mean"))
df_test.to_csv("NY2020;D2;bound=1500;0322.csv")

# Simulation

In [465]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_sample = df_w2
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
# df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
# df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
df_matched = df_matched[df_matched["yearapproved"] == 2020]
# df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
# df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020]
    

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [506]:
df_matched

,lender_id,loan_id,var1,var2,var3,var4,USPS_ZIP_PREF_CITY,USPS_ZIP_PREF_STATE,yearapproved
7,5051.0,97252.0,0.0,9.774425,0.091452,0.045413,AGAWAM,MA,2020.0
25,4081.0,78177.0,0.0,225.861810,0.091452,-0.070219,AGAWAM,MA,2020.0
27,2605.0,27917.0,0.0,0.000000,0.184930,-0.347326,AGAWAM,MA,2020.0
42,1339.0,31464.0,0.0,15.714982,-0.078141,-0.070219,AMHERST,MA,2020.0
69,2129.0,40976.0,0.0,12.762016,0.014364,-0.070219,AMHERST,MA,2020.0
...,...,...,...,...,...,...,...,...,...
4117064,1631.0,34115.0,0.0,0.000000,-0.024180,0.045413,KETCHIKAN,AK,2020.0
4117067,1631.0,34117.0,0.0,0.000000,-0.024180,0.045413,KETCHIKAN,AK,2020.0
4117069,1631.0,34122.0,0.0,65.689996,-0.139812,0.045413,CRAIG,AK,2020.0
4117070,1631.0,34131.0,0.0,61.713988,-0.139812,-0.070219,WRANGELL,AK,2020.0


In [503]:
df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "MA"]["loan_id"].unique().shape

(2340,)

In [504]:
df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "MA"]["loan_id"].unique().shape

(2319,)

In [514]:
# Proportionate Sampling
# Much more quicker than for loops
'''
group_keys parameter in groupby comes handy during apply operations that 
creates an additional index column corresponding to the grouped columns[group_keys=True] 
and eliminates in the case[group_keys=False] especially during the case when trying to 
perform operations on individual columns.
'''
df_sampling = df_matched.groupby("USPS_ZIP_PREF_STATE", group_keys = False).apply(lambda x: x.sample(frac = 0.1))
# df_sampling[df_sampling["USPS_ZIP_PREF_STATE"] == "MA"]
loan_id = df_sampling["loan_id"].unique()

In [515]:
loan_id.shape

(8329,)

In [493]:
loan_id = np.array([])
for item in df_sample["USPS_ZIP_PREF_STATE"].unique():
    loan_id_state = df_matched.append(df_unmatched)[df_matched.append(df_unmatched)["USPS_ZIP_PREF_STATE"] == item]["loan_id"].unique() 
    sample_loan_id = np.random.choice(loan_id_state, round(loan_id_state.shape[0] * 0.1), replace = False)
    loan_id = np.append(loan_id, sample_loan_id)

In [495]:
loan_id.shape

(8329,)

In [461]:
# num is the # of random samples(or the # of simulations)
# ratio is the random sampling ratio, which is 10% here
def Simulation(num, ratio, df_matched, df_unmatched):
    df_result = pd.DataFrame()
    for i in range(0, num):
        # df_matched_sample = Subsample(df_matched, ratio)
        # df_unmatched_sample = Subsample(df_unmatched, ratio)
        
        ##### Use with only one state #####
        # loan_id in both df_matched and df_unmatched
        # loan_id = df_matched.append(df_unmatched)["loan_id"].unique() 
        ##### Use with all states #####
        
        # 不好意思，这个loan id抽样还是不对，先按只是从matched里抽。
        # 如果不对再加unmatched。我想了下，逻辑上应该可以只从matched里开始
        # 你在开头记录一下~这个地方有点不确定~~
        
        df_sampling = df_matched.groupby("USPS_ZIP_PREF_STATE", group_keys = False).apply(lambda x: x.sample(frac = ratio))
        sample_loan_id = df_sampling["loan_id"].unique()
            
        df_unmatched_sample = df_unmatched.loc[df_unmatched["loan_id"].isin(sample_loan_id)]
        df_matched_sample = df_matched.loc[df_matched["loan_id"].isin(sample_loan_id)]
       
        df_keep = Exchange_pairs(df_matched_sample, df_unmatched_sample)
        # bounds = [(1, 1.000000001), (-100, 100), (-100, 100), (-100, 100), (-100, 100)] # fix beta_1 = 1
    
        bounds = [(-100, 100), (-1.000000001, -1), (-100, 100), (-100, 100)]
        # bounds = [(-500, 500), (-1.000000001, -1), (-500, 500), (-500, 500)]
        # bounds = [(-1000, 1000), (-1.000000001, -1), (-1000, 1000), (-1000, 1000)]
        def objectfunc(beta, df = df_keep):
            return -sum(df.dot(beta) >=0 )
        result = differential_evolution(objectfunc, bounds)
        
        r = np.append(result.x, round(-result.fun))
        
        df_result = df_result.append([list(r)], ignore_index = True)
        # print(i)
    # print("for loop finished")
    df_result.columns = ["Beta_1", "Beta_2", "Beta_3", "Beta_4", "Number of of inequalities satisfied"]
    
    num_mean = df_result.mean()
    num_max = df_result["Number of of inequalities satisfied"].max()
    df_result[df_result["Number of of inequalities satisfied"] == num_max]
    # df_test = df_result
    df_test = pd.DataFrame()
    df_test = df_test.append(df_result[df_result["Number of of inequalities satisfied"] == num_max])
    df_test = df_test.append(df_result.mean(axis = 0).rename("mean"))
    
    df_test.index = pd.Series(["argmax", "mean"])
    
    '''
    print("The 5% quantile of parameters are")
    print(df_result.quantile(0.05))
    print("The 95% quantile of parameters are")
    print(df_result.quantile(0.95))
    '''
    
    return df_test



In [ ]:
t1 = time.time()
df_simulation = pd.DataFrame()

for j in range(0, 1000):
    if j%10 == 0:
        print(j)
    df_result = Simulation(100, 0.1, df_matched, df_unmatched)
    df_simulation = df_simulation.append(df_result)

t2 = time.time()
print("Simulation time: ", t2-t1)


0


/opt/conda/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


10


In [448]:
df_simulation

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
argmax,931.518882,-1000.0,-722.851281,590.823406,64854.00
mean,860.448619,-1000.0,-5.402484,680.662903,57239.36
argmax,542.393468,-1000.0,-897.886489,842.916933,64017.00
mean,860.278481,-1000.0,-8.814108,618.555602,56852.69
argmax,982.692821,-1000.0,-33.739912,705.520942,66582.00
...,...,...,...,...,...
mean,874.016324,-1000.0,-54.484276,645.871966,56763.81
argmax,953.382312,-1000.0,-176.418938,872.329123,65967.00
mean,862.904128,-1000.0,-34.642572,714.859121,57130.28
argmax,821.469326,-1000.0,378.287147,872.966397,65423.00


In [419]:
df_simulation.to_csv("Simulation_temp0315.csv")

In [277]:
df_simulation.to_csv("NY2020;Simulation;D2;2;0227.csv")

In [276]:
df_simulation.loc["argmax"]

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
argmax,364.491321,-500.0,-426.703895,464.184568,65333.0
argmax,423.154410,-500.0,72.384652,467.904722,63338.0
argmax,428.245742,-500.0,151.333138,347.970700,63592.0
argmax,430.610743,-500.0,-498.186426,484.522868,63262.0
argmax,406.027309,-500.0,127.161653,488.479870,68447.0
...,...,...,...,...,...
argmax,453.618612,-500.0,-377.629249,419.641761,66357.0
argmax,421.343776,-500.0,-199.793932,448.088632,64853.0
argmax,470.253521,-500.0,-183.742406,362.329910,65539.0
argmax,414.409256,-500.0,-211.800499,407.769768,67160.0


1)N of inequ这里，排名，取第25名，然后找到对应的beta。
2)对于每个beta分开取的，直接用quantile

In [449]:
df_simulation = pd.read_csv("NY2020;Result;D2;2;0227.csv")

In [450]:
df_simulation

,Unnamed: 0,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
0,0.01,178.360669,-500.0,-497.128301,-432.366637,62743.8400
1,0.05,294.270322,-500.0,-481.382506,1.499188,63326.8000
2,0.1,339.603245,-500.0,-464.139620,180.797222,63786.0000
3,0.9,491.971869,-500.0,381.411080,488.058813,67802.5000
4,0.95,495.199450,-500.0,442.711485,494.175099,68509.0000
5,0.99,498.960944,-500.0,489.474245,498.217255,69905.4500
6,0.01,411.201943,-500.0,-107.248631,292.098566,56146.9185
7,0.05,417.613835,-500.0,-84.520545,306.989801,56442.2075
8,0.1,420.614219,-500.0,-72.662028,313.835785,56541.7700
9,0.9,439.647638,-500.0,6.938079,364.937431,57372.8840


In [278]:
df_output = pd.DataFrame()
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.01))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.05))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.10))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.90))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.95))
df_output = df_output.append(df_simulation.loc["argmax"].quantile(0.99))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.01))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.05))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.10))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.90))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.95))
df_output = df_output.append(df_simulation.loc["mean"].quantile(0.99))
df_output

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
0.01,178.360669,-500.0,-497.128301,-432.366637,62743.8400
0.05,294.270322,-500.0,-481.382506,1.499188,63326.8000
0.10,339.603245,-500.0,-464.139620,180.797222,63786.0000
0.90,491.971869,-500.0,381.411080,488.058813,67802.5000
0.95,495.199450,-500.0,442.711485,494.175099,68509.0000
0.99,498.960944,-500.0,489.474245,498.217255,69905.4500
0.01,411.201943,-500.0,-107.248631,292.098566,56146.9185
0.05,417.613835,-500.0,-84.520545,306.989801,56442.2075
0.10,420.614219,-500.0,-72.662028,313.835785,56541.7700
0.90,439.647638,-500.0,6.938079,364.937431,57372.8840


In [279]:
df_argmax = df_simulation.loc["argmax"]
df_mean = df_simulation.loc["mean"]
df1 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[9]]
df_output = df_output.append(df1)
df2 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[49]]
df_output = df_output.append(df2)
df3 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[99]]
df_output = df_output.append(df3)
df4 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[899]]
df_output = df_output.append(df4)
df5 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[949]]
df_output = df_output.append(df5)
df6 = df_argmax[df_argmax["Number of of inequalities satisfied"] == df_argmax["Number of of inequalities satisfied"].sort_values()[989]]
df_output = df_output.append(df6)

In [280]:
df1 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[9]]
df_output = df_output.append(df1)
df2 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[49]]
df_output = df_output.append(df2)
df3 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[99]]
df_output = df_output.append(df3)
df4 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[899]]
df_output = df_output.append(df4)
df5 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[949]]
df_output = df_output.append(df5)
df6 = df_mean[df_mean["Number of of inequalities satisfied"] == df_mean["Number of of inequalities satisfied"].sort_values()[989]]
df_output = df_output.append(df6)

In [281]:
df_simulation.to_csv("NY2020;Simulation;D2;2;0227.csv")
df_output.to_csv("NY2020;Result;D2;2;0227.csv")

In [239]:
df_output

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
0.01,43.029013,-100.0,-98.811193,-89.154596,62929.5200
0.05,60.253665,-100.0,-95.375405,-25.376446,63380.9500
0.1,70.884937,-100.0,-91.172936,25.475582,63770.4000
0.9,98.445446,-100.0,74.612140,97.014122,67810.4000
0.95,99.106805,-100.0,90.800381,98.265524,68543.1000
0.99,99.762228,-100.0,98.456109,99.518228,70371.2700
0.01,82.372831,-100.0,-21.008522,57.618076,56132.3445
0.05,83.478639,-100.0,-16.817538,61.115666,56454.4830
0.1,84.064256,-100.0,-14.720521,62.500499,56557.8710
0.9,87.800807,-100.0,1.309006,73.357585,57422.2680


In [258]:
df_simulation1 = df_simulation

In [264]:
df_simulation2 = df_simulation

In [267]:
df_simulation3 = df_simulation

In [116]:
print("The 10% quantile of parameters are")
print(df_simulation.loc["argmax"].quantile(0.10))
print("The 90% quantile of parameters are")
print(df_simulation.loc["argmax"].quantile(0.90))

The 10% quantile of parameters are
Beta_1                                   358.420266
Beta_2                                  -500.000000
Beta_3                                  -468.969329
Beta_4                                   153.651074
Number of of inequalities satisfied    63875.300000
Name: 0.1, dtype: float64
The 90% quantile of parameters are
Beta_1                                   492.356307
Beta_2                                  -500.000000
Beta_3                                   388.880781
Beta_4                                   485.522528
Number of of inequalities satisfied    67904.900000
Name: 0.9, dtype: float64


In [445]:
a = [1,2,3,4,5,6,7,8,9,10]

In [446]:
np.quantile(a, 0.555)

5.995

In [447]:
df_simulation

,Beta_1,Beta_2,Beta_3,Beta_4,Number of of inequalities satisfied
argmax,931.518882,-1000.0,-722.851281,590.823406,64854.00
mean,860.448619,-1000.0,-5.402484,680.662903,57239.36
argmax,542.393468,-1000.0,-897.886489,842.916933,64017.00
mean,860.278481,-1000.0,-8.814108,618.555602,56852.69
argmax,982.692821,-1000.0,-33.739912,705.520942,66582.00
...,...,...,...,...,...
mean,874.016324,-1000.0,-54.484276,645.871966,56763.81
argmax,953.382312,-1000.0,-176.418938,872.329123,65967.00
mean,862.904128,-1000.0,-34.642572,714.859121,57130.28
argmax,821.469326,-1000.0,378.287147,872.966397,65423.00


# CounterFactual

In [37]:
df_beta = pd.read_csv("NY2020;Result;D2;2;0227.csv").iloc[:, 1:5]
df_beta.head()

,Beta_1,Beta_2,Beta_3,Beta_4
0,178.360669,-500.0,-497.128301,-432.366637
1,294.270322,-500.0,-481.382506,1.499188
2,339.603245,-500.0,-464.139620,180.797222
3,491.971869,-500.0,381.411080,488.058813
4,495.199450,-500.0,442.711485,494.175099


In [421]:
df_org = pd.read_stata('sample_org.dta')
df_org['var1'] = df_org['Relationship_Dum'] - df_org['Relationship_Dum'].mean(axis = 0)
df_org['var2'] = df_org['mi_to_zcta5'] - df_org['mi_to_zcta5'].mean(axis = 0)
df_org['var3'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['rating_avg'] - df_org['rating_avg'].mean(axis = 0))
df_org['var4'] = (df_org['FinTechIndicator'] - df_org['FinTechIndicator'].mean(axis = 0)) * (df_org['minority_yelp'] - df_org['minority_yelp'].mean(axis = 0))

'''
df_w1 = pd.DataFrame()
df_w1 = df_org[['lender_id', 'loan_id', 'match']]
df_w1.loc[:, 'var1'] = df_org['var1'].values
df_w1.loc[:, 'var2'] = df_org['var2'].values
df_w1.loc[:, 'var3'] = df_org['var3'].values
df_w1.loc[:, 'var4'] = df_org['var4'].values
df_w1.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values
'''

df_w2 = pd.DataFrame()
df_w2 = df_org[['lender_id', 'loan_id', 'match']]
df_w2.loc[:, 'var1'] = df_org['Relationship_Dum'].values
df_w2.loc[:, 'var2'] = df_org['mi_to_zcta5'].values
df_w2.loc[:, 'var3'] = df_org['var3'].values
df_w2.loc[:, 'var4'] = df_org['var4'].values
df_w2.loc[:, ('USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved')] = df_org[['USPS_ZIP_PREF_CITY', 'USPS_ZIP_PREF_STATE', 'yearapproved']].values

df_sample = df_w2
'''
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 
df_matched = df_matched[df_matched["USPS_ZIP_PREF_STATE"] == "NY"]
df_matched = df_matched[df_matched["yearapproved"] == 2020].iloc[:,: 6]
df_unmatched = df_unmatched[df_unmatched["USPS_ZIP_PREF_STATE"] == "NY"]
df_unmatched = df_unmatched[df_unmatched["yearapproved"] == 2020].iloc[:,: 6]
'''
# df_sample.drop(["match"], axis = 1, inplace = True) 
df_sample = df_sample[df_sample["USPS_ZIP_PREF_STATE"] == "NY"]
df_sample = df_sample[df_sample["yearapproved"] == 2020].iloc[:,: 7]
df_matched = df_sample[df_sample["match"] == 1]
df_unmatched = df_sample[df_sample["match"] == 0]
df_matched.drop(["match"], axis = 1, inplace = True) 
df_unmatched.drop(["match"], axis = 1, inplace = True) 

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/opt/conda/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [426]:
df_org_sample = df_org[df_org["USPS_ZIP_PREF_STATE"] == "NY"]
df_org_sample = df_org_sample[df_org_sample["yearapproved"] == 2020]
df_org_sample
df_sample["FinTechIndicator"] = df_org_sample["FinTechIndicator"]
df_sample["MinorityIndicator"] = df_org_sample["minority_yelp"]
df_matched["FinTechIndicator"] = df_org_sample[df_org_sample["match"] == 1]["FinTechIndicator"]
df_matched["MinorityIndicator"] = df_org_sample[df_org_sample["match"] == 1]["minority_yelp"]

/tmp/ipykernel_907/2952620905.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matched["FinTechIndicator"] = df_org_sample[df_org_sample["match"] == 1]["FinTechIndicator"]
/tmp/ipykernel_907/2952620905.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matched["MinorityIndicator"] = df_org_sample[df_org_sample["match"] == 1]["minority_yelp"]


In [434]:
df_pred = df_sample.copy()
df_pred["FinTechIndicator"] = df_org_sample["FinTechIndicator"]
df_pred["MinorityIndicator"] = df_org_sample["minority_yelp"]

In [435]:
df_obs = pd.DataFrame()
df_obs.loc["obs", "% of minority using fintech"] = df_matched[df_matched["MinorityIndicator"] == 1]["FinTechIndicator"].sum()/df_matched.shape[0]
df_obs.loc["obs", "% of non-minority using fintech"] = df_matched[df_matched["MinorityIndicator"] == 0]["FinTechIndicator"].sum()/df_matched.shape[0]
df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["obs", "% of minority using fintech"] - df_obs.loc["obs", "% of non-minority using fintech"]
df_obs

,% of minority using fintech,% of non-minority using fintech,% of minority using fintech - % of non-minority using fintech
obs,0.087769,0.079046,0.008724


In [436]:
betas = np.array(df_beta.loc[i].values).reshape(4,1)
df_pred["ShutoffBeta4"] = df_pred.iloc[:, 2:5].dot(betas[0:3])
b = np.array([[1],[1],[1],[-1]]) # shape = (4,1)
# df_pred["pred"] = df_pred.iloc[:, 2:6].dot(betas) >= 0
df_pred["ReverseBeta4"] = df_pred.iloc[:, 2:6].dot(np.multiply(betas, b)) # Hadamard Product
df_pred["ShutoffBeta1"] = df_pred.iloc[:, 3:6].dot(betas[1:4])
df_pred["ShutoffBeta2"] = df_pred.iloc[:, [2, 4, 5]].dot(betas[[0, 2, 3]])
df_pred["ShutoffBeta3"] = df_pred.iloc[:, [2, 3, 5]].dot(betas[[0, 1, 3]])

In [437]:
# Calculate lender's quota 
df_quota = pd.DataFrame(df_pred["lender_id"].value_counts())
df_quota.columns = ["quota"]
df_quota["used"] = 0
df_quota

,quota,used
329.0,4288,0
2605.0,4188,0
2572.0,4175,0
1224.0,4014,0
5000.0,3937,0
...,...,...
2665.0,1,0
2312.0,1,0
3222.0,1,0
4797.0,1,0


In [442]:
df_pred

,lender_id,loan_id,match,var1,var2,var3,var4,FinTechIndicator,MinorityIndicator,ShutoffBeta4,ReverseBeta4,ShutoffBeta1,ShutoffBeta2,ShutoffBeta3
515955,2605.0,52303.0,1.0,0.0,0.000000,-2.468174,0.537042,1,1.0,1.783607e+02,-8.887953e+02,1.067156e+03,1.245517e+03,1245.516658
568668,3923.0,73892.0,1.0,0.0,0.000000,1.069299,-0.347326,1,0.0,1.783607e+02,6.406897e+02,-4.623290e+02,-2.839683e+02,-283.968329
591213,3923.0,74230.0,1.0,0.0,0.000000,1.069299,-0.347326,1,0.0,1.783607e+02,6.406897e+02,-4.623290e+02,-2.839683e+02,-283.968329
383881,2605.0,27889.0,1.0,0.0,0.000000,-1.289016,0.537042,1,1.0,1.783607e+02,-3.789670e+02,5.573277e+02,7.356884e+02,735.688359
473642,295.0,4429.0,1.0,0.0,0.000000,-1.289016,0.537042,1,1.0,1.783607e+02,-3.789670e+02,5.573277e+02,7.356884e+02,735.688359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354540,1792.0,66800.0,0.0,0.0,4961.197628,-0.101268,-0.070219,0,1.0,-2.466352e+06,-2.466396e+06,-2.466308e+06,-2.466308e+06,43.784802
397083,1792.0,50338.0,0.0,0.0,4967.830913,-0.139812,0.045413,0,0.0,-2.469649e+06,-2.469710e+06,-2.469589e+06,-2.469589e+06,60.449940
397082,1792.0,95118.0,0.0,0.0,4967.830913,-0.081996,0.045413,0,0.0,-2.469649e+06,-2.469685e+06,-2.469614e+06,-2.469614e+06,35.452245
397080,1792.0,34652.0,0.0,0.0,4967.830913,0.074933,0.045413,0,0.0,-2.469649e+06,-2.469617e+06,-2.469682e+06,-2.469682e+06,-32.398646


In [443]:
t1 = time.time()
for item in ["ShutoffBeta4", "ReverseBeta4", "ShutoffBeta1", "ShutoffBeta2", "ShutoffBeta3"]:
    
    # Sort matching values by betas
    df_pred.sort_values(by = [item], axis = 0, ascending = False, inplace = True)
    
    # Calculate lender's quota and Initialize df_quota per loop
    df_quota = pd.DataFrame(df_pred["lender_id"].value_counts())
    df_quota.columns = ["quota"]
    df_quota["used"] = 0 # 
    
    # Now do the assignment
    for index, row in df_pred.iterrows():
        if df_quota.loc[row["lender_id"], "used"] < df_quota.loc[row["lender_id"], "quota"]:
            df_quota.loc[row["lender_id"], "used"] += 1
            # Assign borrower, then delete the borrower
            df_pred.drop(df_pred[df_pred["loan_id"] == row["loan_id"]].index.delete(0), inplace = True)
        else:
            # Delete lender since quota is full
            df_pred.drop(df_pred[df_pred["lender_id"] == row["lender_id"]].index, inplace = True)
    
    # Save the results to df_obs
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred[item] >= 0)]
    df_obs.loc[item, "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred[item] >= 0)]
    df_obs.loc[item, "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc[item, "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc[item, "% of minority using fintech"] - df_obs.loc[item, "% of non-minority using fintech"] 
    df_obs.loc[item, "Obs. – Counter."] = df_obs.loc[item, "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    t3 = time.time()
    print(item + "Assign time: ", t3 - t1)  

t2 = time.time()
print("Total assign time: ", t2 - t1)     

ShutoffBeta4Assign time:  1734.7824125289917
ReverseBeta4Assign time:  1759.879518032074
ShutoffBeta1Assign time:  1785.284848690033
ShutoffBeta2Assign time:  1810.9781539440155
ShutoffBeta3Assign time:  1836.1676132678986
Total assign time:  1836.1678531169891


In [444]:
df_obs

,% of minority using fintech,% of non-minority using fintech,% of minority using fintech - % of non-minority using fintech,Obs. – Counter.
obs,0.087769,0.079046,0.008724,NaN
ShutoffBeta4,0.905191,0.882407,0.022783,0.014060
ReverseBeta4,0.946942,0.904863,0.042080,0.033356
ShutoffBeta1,0.835458,0.811749,0.023709,0.014985
ShutoffBeta2,0.849549,0.804067,0.045482,0.036759
ShutoffBeta3,0.823129,0.669231,0.153898,0.145175


# The following part is my draft, I will delete them later

In [375]:
# Assign time:  3077.200585126877
df_pred1 = df_pred.copy()

In [ ]:
df_pred1

In [412]:
# Error
# Assign time:  3343.7934064865112
df_pred4 = df_pred.copy()

In [ ]:
df_pred4

In [373]:
# Shut off Beta4
df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta4"] >= 0)]
df_obs.loc["ShutoffBeta4", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta4"] >= 0)]
df_obs.loc["ShutoffBeta4", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
df_obs.loc["ShutoffBeta4", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta4", "% of minority using fintech"] - df_obs.loc["ShutoffBeta4", "% of non-minority using fintech"] 
df_obs.loc["ShutoffBeta4", "Obs. – Counter."] = df_obs.loc["ShutoffBeta4", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 

In [205]:
# for i in range(0, df_beta.shape[0]):
for i in range(0, 1):
    betas = np.array(df_beta.loc[i].values).reshape(4,1)
    b = np.array([[1],[1],[1],[-1]]) # shape = (4,1)
    # df_pred["pred"] = df_pred.iloc[:, 2:6].dot(betas) >= 0
    '''
    df_pred["ShutoffBeta4"] = df_pred.iloc[:, 2:5].dot(betas[0:3]) >= 0
    df_pred["ReverseBeta4"] = df_pred.iloc[:, 2:6].dot(np.multiply(betas, b)) >= 0 # Hadamard Product
    df_pred["ShutoffBeta1"] = df_pred.iloc[:, 3:6].dot(betas[1:4]) >= 0
    df_pred["ShutoffBeta2"] = df_pred.iloc[:, [2, 4, 5]].dot(betas[[0, 2, 3]]) >= 0
    df_pred["ShutoffBeta3"] = df_pred.iloc[:, [2, 3, 5]].dot(betas[[0, 1, 3]]) >= 0
    '''
    
    # Shut off Beta4
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta4"] >= 0)]
    df_obs.loc["ShutoffBeta4", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta4"] >= 0)]
    df_obs.loc["ShutoffBeta4", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ShutoffBeta4", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta4", "% of minority using fintech"] - df_obs.loc["ShutoffBeta4", "% of non-minority using fintech"] 
    df_obs.loc["ShutoffBeta4", "Obs. – Counter."] = df_obs.loc["ShutoffBeta4", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    # Reverse Beta4
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ReverseBeta4"] >= 0)]
    df_obs.loc["ReverseBeta4", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ReverseBeta4"] >= 0)]
    df_obs.loc["ReverseBeta4", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ReverseBeta4", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ReverseBeta4", "% of minority using fintech"] - df_obs.loc["ReverseBeta4", "% of non-minority using fintech"] 
    df_obs.loc["ReverseBeta4", "Obs. – Counter."] = df_obs.loc["ReverseBeta4", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    # Shut off Beta1
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta1"] >= 0)]
    df_obs.loc["ShutoffBeta1", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta1"] >= 0)]
    df_obs.loc["ShutoffBeta1", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ShutoffBeta1", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta1", "% of minority using fintech"] - df_obs.loc["ShutoffBeta1", "% of non-minority using fintech"] 
    df_obs.loc["ShutoffBeta1", "Obs. – Counter."] = df_obs.loc["ShutoffBeta1", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    # Shut off Beta2
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta2"] >= 0)]
    df_obs.loc["ShutoffBeta2", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta2"] >= 0)]
    df_obs.loc["ShutoffBeta2", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ShutoffBeta2", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta2", "% of minority using fintech"] - df_obs.loc["ShutoffBeta2", "% of non-minority using fintech"] 
    df_obs.loc["ShutoffBeta2", "Obs. – Counter."] = df_obs.loc["ShutoffBeta2", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    
    # Shut off Beta3
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 1) & (df_pred["ShutoffBeta3"] >= 0)]
    df_obs.loc["ShutoffBeta3", "% of minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_temp = df_pred[(df_pred["MinorityIndicator"] == 0) & (df_pred["ShutoffBeta3"] >= 0)]
    df_obs.loc["ShutoffBeta3", "% of non-minority using fintech"] = df_temp["FinTechIndicator"].sum()/df_temp.shape[0]
    df_obs.loc["ShutoffBeta3", "% of minority using fintech - % of non-minority using fintech"] = df_obs.loc["ShutoffBeta3", "% of minority using fintech"] - df_obs.loc["ShutoffBeta3", "% of non-minority using fintech"] 
    df_obs.loc["ShutoffBeta3", "Obs. – Counter."] = df_obs.loc["ShutoffBeta3", "% of minority using fintech - % of non-minority using fintech"] - df_obs.loc["obs", "% of minority using fintech - % of non-minority using fintech"] 
    